# 영어 Word2Vec 만들기

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
import re
from urllib.request import urlretrieve # 인터넷에서 데이터를 가져올때
import zipfile
from lxml import etree          # 가져올 데이터가 XML이라 XML parser가 필요함
from nltk.tokenize import word_tokenize, sent_tokenize

## 훈련데이터 이해하기

In [11]:
# 데이터 다운로드
urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f9db69ef150>)

In [12]:
!ls -l ted*

-rw-r--r-- 1 root root 74533638 Jan 25 01:05 ted_en-20160408.xml


In [14]:
!more ted*
# more는 ted*의 파일 내용을 한페이지씩 읽는다 
# 그만 읽을땐 q  스페이스를 치면 계속.  한줄은 r  

<?xml version="1.0" encoding="UTF-8"?>
<xml language="en"><file id="1">
  <head>
    <url>http://www.ted.com/talks/knut_haanaes_two_reasons_companies_fail_and_ho
w_to_avoid_them</url>
    <pagesize>72832</pagesize>
    <dtime>Fri Apr 01 00:57:03 CEST 2016</dtime>
    <encoding>UTF-8</encoding>
    <content-type>text/html; charset=utf-8</content-type>
    <keywords>talks, business, creativity, curiosity, goal-setting, innovation, 
motivation, potential, success, work</keywords>
    <speaker>Knut Haanaes</speaker>
    <talkid>2470</talkid>
    <videourl>http://download.ted.com/talks/KnutHaanaes_2015S.mp4</videourl>
    <videopath>talks/KnutHaanaes_2015S.mp4</videopath>
    <date>2015/06/30</date>
    <title>Knut Haanaes: Two reasons companies fail -- and how to avoid them</ti
tle>
    <description>TED Talk Subtitles and Transcript: Is it possible to run a comp
any and reinvent it at the same time? For business strategist Knut Haanaes, the 
ability to innovate after becoming successful is

## 훈련 데이터 전처리하기

In [15]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

In [17]:
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [18]:
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [19]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

In [20]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

In [21]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [25]:
print(f'총 샘플의 개수 : {len(result):,d}') # d 천단위로 구둣점 추가

총 샘플의 개수 : 273,424


In [26]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## Word2Vec 훈련시키기

    size = 워드 벡터의 특징 값. 즉, 임베딩 된   벡터의 차원.
    window = 컨텍스트 윈도우 크기
    min_count = 단어 최소 빈도 수 제한 (빈도가  적은 단어들은 학습하지 않는다.)
    workers = 학습을 위한 프로세스 수
    sg = 0은 CBOW, 1은 Skip-gram.

In [28]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [30]:
%time model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

CPU times: user 1min 5s, sys: 352 ms, total: 1min 5s
Wall time: 37.7 s


In [31]:
# man과 자주 나오는(similar) 단어 10개 
model.wv.most_similar("man")

[('woman', 0.8662915825843811),
 ('guy', 0.8036783933639526),
 ('girl', 0.7517255544662476),
 ('soldier', 0.7456632256507874),
 ('lady', 0.7443767786026001),
 ('boy', 0.7363971471786499),
 ('kid', 0.7207381129264832),
 ('gentleman', 0.7200431823730469),
 ('poet', 0.6770505905151367),
 ('king', 0.6721272468566895)]

In [32]:
# man - boy + girl --> woman
model.wv.most_similar(positive=['man', 'girl'], negative=['boy'], topn=3)

[('woman', 0.8580714464187622),
 ('guy', 0.7282146215438843),
 ('soldier', 0.7161544561386108)]

## 모델 저정하고 로드하기

In [33]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장

In [34]:
!ls -l

total 3656732
-rw-r--r-- 1 root root   25689555 Jan 25 01:30 eng_w2v
-rw-r--r-- 1 root root 3644258522 Mar  5  2015 GoogleNews-vectors-negative300.bin
drwxr-xr-x 1 root root       4096 Jan  7 14:33 sample_data
-rw-r--r-- 1 root root   74533638 Jan 25 01:05 ted_en-20160408.xml


In [35]:
from gensim.models import KeyedVectors
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드
loaded_model.most_similar("man")

[('woman', 0.8662915825843811),
 ('guy', 0.8036783933639526),
 ('girl', 0.7517255544662476),
 ('soldier', 0.7456632256507874),
 ('lady', 0.7443767786026001),
 ('boy', 0.7363971471786499),
 ('kid', 0.7207381129264832),
 ('gentleman', 0.7200431823730469),
 ('poet', 0.6770505905151367),
 ('king', 0.6721272468566895)]